In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

In [3]:
task1 = open("/content/drive/My Drive/NLP/dialog-bAbI-tasks_1_/dialog-bAbI-tasks/dialog-babi-task1-API-calls-trn.txt",'r')
task2 = open("/content/drive/My Drive/NLP/dialog-bAbI-tasks_1_/dialog-bAbI-tasks/dialog-babi-task2-API-refine-trn.txt",'r')
files = [task1,task2]

In [4]:
questions = list()
answers = list()

In [5]:
ind = 0
for file in files:
    print("inside file")
    data = file.read()
    lines = data.split("\n")
    for i in lines:
        if len(i)<1:
            continue
        inout=i.split("\t")
        if(len(inout)==1):
            out = inout[0]
            out = out.split(out.split()[0])
            answers[ind-1] = answers[ind-1]+ "\n" +out[1]
        else:
            inp,out = inout[0],inout[1]
            inp=inp.split(inp.split()[0])
            if(inp[1]!=' <SILENCE>'):
                questions.append(inp[1])
                answers.append(out)
                ind+=1
            else:
                answers[ind-1] = answers[ind-1]+', '+out 
    file.close()

inside file
inside file


In [6]:
save = open("/content/drive/My Drive/NLP/dialog-bAbI-tasks_1_/attention_data.txt","w")
for i in range(len(questions)):
  save.write(questions[i].strip()+"\t"+answers[i].strip()+"\n")
save.close()

In [7]:
path_to_file = "/content/drive/My Drive/NLP/dialog-bAbI-tasks_1_/attention_data.txt"

In [11]:
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())

  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  w = w.strip()

  w = '<start> ' + w + ' <end>'
  return w

In [15]:
qn_sentence = u"good morning"
ans_sentence = u"hello what can i help you with today"
print(preprocess_sentence(qn_sentence))
print(preprocess_sentence(ans_sentence).encode('utf-8'))

<start> good morning <end>
b'<start> hello what can i help you with today <end>'


In [16]:
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]

  return zip(*word_pairs)

In [17]:
qn, ans = create_dataset(path_to_file, None)
print(qn[-1])
print(ans[-1])

<start> thanks <end>
<start> you re welcome <end>


In [25]:
def tokenize(stmt):
  stmt_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  stmt_tokenizer.fit_on_texts(stmt)

  tensor = stmt_tokenizer.texts_to_sequences(stmt)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, stmt_tokenizer

In [26]:
def load_dataset(path, num_examples=None):
  inp_stmt,targ_stmt  = create_dataset(path, num_examples)

  input_tensor, inp_stmt_tokenizer = tokenize(inp_stmt)
  target_tensor, targ_stmt_tokenizer = tokenize(targ_stmt)

  return input_tensor, target_tensor, inp_stmt_tokenizer, targ_stmt_tokenizer

In [27]:
num_examples = 10522
input_tensor, target_tensor, inp_stmt, targ_stmt = load_dataset(path_to_file, num_examples)

max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [28]:
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.1)

print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

9469 9469 1053 1053


In [29]:
def convert(stmt, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, stmt.index_word[t]))

In [30]:
print ("Input; index to word mapping")
convert(inp_stmt, input_tensor_train[0])
print ()
print ("Output; index to word mapping")
convert(targ_stmt, target_tensor_train[0])

Input; index to word mapping
1 ----> <start>
5 ----> for
36 ----> six
27 ----> please
2 ----> <end>

Output; index to word mapping
1 ----> <start>
59 ----> which
60 ----> price
61 ----> range
62 ----> are
63 ----> looking
6 ----> for
2 ----> <end>


In [31]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_stmt.word_index)+1
vocab_tar_size = len(targ_stmt.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [32]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 22]), TensorShape([64, 23]))

In [33]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [34]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 22, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [35]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    query_with_time_axis = tf.expand_dims(query, 1)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    attention_weights = tf.nn.softmax(score, axis=1)

    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [36]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 22, 1)


In [43]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    context_vector, attention_weights = self.attention(hidden, enc_output)
    x = self.embedding(x)

    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
    output, state = self.gru(x)

    output = tf.reshape(output, (-1, output.shape[2]))

    x = self.fc(output)

    return x, state, attention_weights

In [44]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 72)


In [45]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [46]:
checkpoint_dir = '/content/drive/My Drive/chatbot/attention_training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [47]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_stmt.word_index['<start>']] * BATCH_SIZE, 1)

    for t in range(1, targ.shape[1]):
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [ ]:
EPOCHS = 15
loss_list = []
epochs_list = []
for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))

  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)
  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  loss_list.append(total_loss / steps_per_epoch)
  epochs_list.append(epoch + 1)
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 2.1090
Epoch 1 Batch 100 Loss 0.1050


In [ ]:
import matplotlib.pyplot as plt
plt.plot(epochs_list,loss_list)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
def evaluate(sentence):

  sentence = preprocess_sentence(sentence)

  inputs = [inp_stmt.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_stmt.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)
    
    attention_weights = tf.reshape(attention_weights, (-1, ))
 
    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_stmt.index_word[predicted_id] + ' '

    if targ_stmt.index_word[predicted_id] == '<end>':
      return result, sentence

    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence

In [ ]:
def reply(sentence):
  result, sentence = evaluate(sentence)

  return str(result)

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
reply("hello")

In [ ]:
print("enter 'close' to close this chat bot\n\n")
res = ""
loc = ""
hmany = ""
price = ""
inp = ""
while(True):
    try:
        inp = input( 'Enter question : ' )
        if(inp=="close"):
            break
        test = nlp(inp)
        for word in test.ents:
            if(word.label_=="NORP"):
                res = word.text
#                 print("food type",word.text)
            if(word.label_=="CARDINAL"):
                hmany = word.text
#                 print("no of people",word.text)
            if(word.label_=="GPE"):
                loc = word.text
#                 print("location",word.text)
        if("moderate" in inp):
            price = "moderate"
        elif("expensive" in inp):
            price = "expensive"
        elif("cheap" in inp):
            price = "cheap"
        decoded_reply = reply(inp)
        if(decoded_reply=="you re welcome <end> " or decoded_reply=="hello what can i help you with today <end> "):
            res = ""
            loc = ""
            hmany = ""
            price = ""
        if ("api call" in decoded_reply):
            l = list(decoded_reply.rsplit("api call",1)[1].strip().split(" "))
            if(res!=l[0] or hmany!=l[2] or loc!=l[1] or price!=l[3]):
                if(res==""):
                    res = l[0]
                if(hmany==""):
                    hmany = l[2]
                if(loc == ""):
                    loc = l[1]
                if(price == ""):
                    price = l[3]
                temp1,temp2 = decoded_reply.rsplit("api call",1)
                decoded_reply = temp1+"api call "+res+" "+loc+" "+hmany+" "+price+" end"
        print( decoded_reply.replace("  "," "))
        
    except KeyError:
        print("sorry i dont know those words!")

In [ ]:
task6 = open("/content/drive/My Drive/NLP/dialog-bAbI-tasks_1_/dialog-bAbI-tasks/dialog-babi-task1-API-calls-tst.txt",'r')
task7 = open("/content/drive/My Drive/NLP/dialog-bAbI-tasks_1_/dialog-bAbI-tasks/dialog-babi-task2-API-refine-tst.txt",'r')
files = [task6,task7]
questions = list()
answers = list()
ind = 0
for file in files:
    print("inside file")
    data = file.read()
    lines = data.split("\n")
    for i in lines:
        if len(i)<1:
            continue
        i = i.replace("_"," ")
        inout=i.split("\t")
        if(len(inout)==1):
            out = inout[0]
            out = out.split(out.split()[0])
            answers[ind-1] = answers[ind-1]+ "\n" +out[1]
        else:
            inp,out = inout[0],inout[1]
            inp=inp.split(inp.split()[0])
            if(inp[1]!=' <SILENCE>'):
                questions.append(inp[1])
                answers.append(out)
                ind+=1
            else:
                answers[ind-1] = answers[ind-1]+', '+out 
    file.close()

In [ ]:
answers_with_tags = list()
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( preprocess_sentence(answers[i]) )
    else:
        questions.pop( i )

In [ ]:
len(questions)
len(answers_with_tags)

In [ ]:
res = ""
loc = ""
hmany = ""
price = ""
inp = ""
var = 0
count = 0
while(var<len(questions)):
    try:
        inp = questions[var]
        if(inp=="close"):
            break
        test = nlp(inp)
        for word in test.ents:
            if(word.label_=="NORP"):
                res = word.text

            if(word.label_=="CARDINAL"):
                hmany = word.text

            if(word.label_=="GPE"):
                loc = word.text

        if("moderate" in inp):
            price = "moderate"
        elif("expensive" in inp):
            price = "expensive"
        elif("cheap" in inp):
            price = "cheap"
        decoded_reply = reply(inp)
        if(decoded_reply=="you re welcome <end> " or reply=="hello what can i help you with today <end> "):
            res = ""
            loc = ""
            hmany = ""
            price = ""
        if ("api call" in decoded_reply):
            l = list(decoded_reply.rsplit("api call",1)[1].strip().split(" "))
            if(res!=l[0] or hmany!=l[2] or loc!=l[1] or price!=l[3]):
                if(res==""):
                    res = l[0]
                if(hmany==""):
                    hmany = l[2]
                if(loc == ""):
                    loc = l[1]
                if(price == ""):
                    price = l[3]
                temp1,temp2 = decoded_reply.rsplit("api call",1)
                decoded_reply = temp1+"api call "+res+" "+loc+" "+hmany+" "+price+" <end>"
        decoded_reply = decoded_reply.replace("  "," ")
        decoded_reply = decoded_reply.replace("<end>","")
        decoded_reply = decoded_reply.strip()
        answers_with_tags[var] = answers_with_tags[var].replace("<end>","")
        answers_with_tags[var] = answers_with_tags[var].replace("<start>","")
        answers_with_tags[var] = answers_with_tags[var].strip()
        if(answers_with_tags[var] == decoded_reply):
            count+=1
        else:
            print(decoded_reply,var)
            print(answers_with_tags[var],var)

    except KeyError:
        print("sorry i dont know those words!")
    var+=1

In [ ]:
print("total = ",len(questions),"\ncorrect = ",count)
print((count*100)/len(questions),"%")